# Python for Webscraping
* SOC 590: Big Data and Population Processes
* 17th October 2016

## Tutorial 3: Webscraping and pre-processing 

In [1]:
import os
import urllib
import webbrowser
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline

In [2]:
url = 'http://www.pro-football-reference.com/years/2015/passing.htm'
webbrowser.open_new_tab(url)

True

In [3]:
# The url we will be scraping
url_2015 = "http://www.pro-football-reference.com/years/2015/passing.htm"

# get the html
html = urllib.request.urlopen(url_2015)

# create the BeautifulSoup object
soup = BeautifulSoup(html, "lxml")

## Scraping the Column Headers

The column headers we need for our `DataFrame` are found in the second row of column headers PFR table.  We will will scrape those and add two additional columns headers for the two additional player page links.

In [4]:
# Extract the necessary values for the column headers from the table
# and store them as a list
column_headers = [th.getText() for th in soup.findAll('th', limit=30)]
column_headers = [s for s in column_headers if len(s) != 0]
column_headers = column_headers[1:]
column_headers

['Tm',
 'Age',
 'Pos',
 'G',
 'GS',
 'QBrec',
 'Cmp',
 'Att',
 'Cmp%',
 'Yds',
 'TD',
 'TD%',
 'Int',
 'Int%',
 'Lng',
 'Y/A',
 'AY/A',
 'Y/C',
 'Y/G',
 'Rate',
 'QBR',
 'Sk',
 'Yds',
 'NY/A',
 'ANY/A',
 'Sk%',
 '4QC',
 'GWD']

In [5]:
len(column_headers)

28

## Scraping the Data

In [6]:
# The data is found within the table rows
# We want the elements from the 3rd row and on
table_rows = soup.find_all("tr")[1:]
table_rows

[<tr><th class="right " csk="1" data-stat="ranker" scope="row">1</th><td class="left " csk="Rivers,Philip" data-append-csv="RivePh00" data-stat="player"><a href="/players/R/RivePh00.htm">Philip Rivers</a></td><td class="left " data-stat="team"><a href="/teams/sdg/2015.htm" title="San Diego Chargers">SDG</a></td><td class="right " data-stat="age">34</td><td class="left " data-stat="pos">QB</td><td class="right " data-stat="g">16</td><td class="right " data-stat="gs">16</td><td class="right " csk="0.25000" data-stat="qb_rec">4-12-0</td><td class="right " data-stat="pass_cmp">437</td><td class="right " data-stat="pass_att">661</td><td class="right " data-stat="pass_cmp_perc">66.1</td><td class="right " data-stat="pass_yds">4792</td><td class="right " data-stat="pass_td">29</td><td class="right " data-stat="pass_td_perc">4.4</td><td class="right " data-stat="pass_int">13</td><td class="right " data-stat="pass_int_perc">2.0</td><td class="right " data-stat="pass_long">80</td><td class="righ

Note that `table_rows` is a list of tag elements.

In [7]:
type(table_rows)

list

In [8]:
table_rows[0] # take a look at the first row

<tr><th class="right " csk="1" data-stat="ranker" scope="row">1</th><td class="left " csk="Rivers,Philip" data-append-csv="RivePh00" data-stat="player"><a href="/players/R/RivePh00.htm">Philip Rivers</a></td><td class="left " data-stat="team"><a href="/teams/sdg/2015.htm" title="San Diego Chargers">SDG</a></td><td class="right " data-stat="age">34</td><td class="left " data-stat="pos">QB</td><td class="right " data-stat="g">16</td><td class="right " data-stat="gs">16</td><td class="right " csk="0.25000" data-stat="qb_rec">4-12-0</td><td class="right " data-stat="pass_cmp">437</td><td class="right " data-stat="pass_att">661</td><td class="right " data-stat="pass_cmp_perc">66.1</td><td class="right " data-stat="pass_yds">4792</td><td class="right " data-stat="pass_td">29</td><td class="right " data-stat="pass_td_perc">4.4</td><td class="right " data-stat="pass_int">13</td><td class="right " data-stat="pass_int_perc">2.0</td><td class="right " data-stat="pass_long">80</td><td class="right

The data we want for each player is found within the the `td` (or table data) elements.  

Below I've created a function that extracts the data we want from `table_rows`.  The comments should walk you through what each part of the function does.

In [9]:
def extract_player_data(table_rows):
    """
    Extract and return the the desired information from the td elements within
    the table rows.
    """
    # create the empty list to store the player data
    player_data = []
    
    for row in table_rows:  # for each row do the following

        # Get the text for each table data (td) element in the row
        # Some player names end with ' HOF', if they do, get the text excluding
        # those last 4 characters,
        # otherwise get all the text data from the table data
        player_list = [td.get_text() for td in row.find_all("td")]
        
        
        # there are some empty table rows, which are the repeated 
        # column headers in the table
        # we skip over those rows and and continue the for loop
        if not player_list:
            continue
            
        # Now append the data to list of data
        player_data.append(player_list)
        
    return player_data

Now we can create a `DataFrame` with the 2016 passing data

In [10]:
# extract the data we want
data = extract_player_data(table_rows)
#column_headers
# and then store it in a DataFrame
#df_2016 = pd.DataFrame(data, columns=column_headers)
df_2016  = pd.DataFrame(data)

In [11]:
data[1]

['Drew Brees',
 'NOR',
 '36',
 'QB',
 '15',
 '15',
 '7-8-0',
 '428',
 '627',
 '68.3',
 '4870',
 '32',
 '5.1',
 '11',
 '1.8',
 '80',
 '7.8',
 '8.0',
 '11.4',
 '324.7',
 '101.0',
 '75.47',
 '31',
 '235',
 '7.04',
 '7.26',
 '4.7',
 '1',
 '2']

In [12]:
df_2016.columns = ['Player_name']+column_headers

In [13]:
df_2016.head()

,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,Philip Rivers,SDG,34,QB,16,16,4-12-0,437,661,66.1,...,299.5,93.8,59.44,40,264,6.46,6.45,5.7,1,2
1,Drew Brees,NOR,36,QB,15,15,7-8-0,428,627,68.3,...,324.7,101.0,75.47,31,235,7.04,7.26,4.7,1,2
2,Tom Brady*,NWE,38,QB,16,16,12-4-0,402,624,64.4,...,298.1,102.2,64.42,38,225,6.87,7.48,5.7,2,2
3,Eli Manning*,NYG,34,QB,16,16,6-10-0,387,618,62.6,...,277.0,93.6,60.46,27,157,6.63,6.74,4.2,1,2
4,Matt Ryan,ATL,30,QB,16,16,8-8-0,407,614,66.3,...,286.9,89.0,61.79,30,203,6.81,6.35,4.7,4,4


### Scraping the Data for All Seasons Since 1967

Scraping the for passing data since 1967 follows is essentially the same process as above, just repeated for each year, using a `for` loop.

As we loop over the years, we will create a `DataFrame` for each year of passing data, and append it to a large list of `DataFrame`s that contains all the passing data.  We will also have a separate list that will contain any errors and the url associated with that error.  This will let us know if there are any issues with our scraper, and which url is causing the error.

In [14]:
# Create an empty list that will contain all the dataframes
# (one dataframe for all passing dataframes)
passing_dfs_list = []

# a list to store any errors that may come up while scraping
errors_list = []

In [15]:
# The url template that we pass in the passing year info
url_template = "http://www.pro-football-reference.com/years/{year}/passing.htm"

# for each year from 1967 to (and including) 2016
for year in range(1967, 2017): 
    
    # Use try/except block to catch and inspect any urls that cause an error
    try:
        # get the passing data url
        url = url_template.format(year=year)

        # get the html
        html = urllib.request.urlopen(url)

        # create the BeautifulSoup object
        soup = BeautifulSoup(html, "lxml") 

        # get the column headers
        column_headers = [th.getText() for th in soup.findAll('th', limit=30)]

        column_headers = [s for s in column_headers if len(s) != 0]
        column_headers = column_headers[1:]
        
        
        #column_headers = ['Player_name']+column_headers
        #print(column_headers)
        

        # select the data from the table 
        table_rows = soup.find_all("tr")[1:]

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)
        #print((column_headers))
        # create the dataframe for the current year's passing data
        year_df = pd.DataFrame(player_data)

    
        # add the year of the pssing data to the dataframe
        year_df.insert(0, "Year", year)
   
        # append the current dataframe to the list of dataframes
        passing_dfs_list.append(year_df)
    
    except Exception as e:
        # Store the url and the error it causes in a list
        error =[url, e] 
        # then append it to the list of errors
        errors_list.append(error)

In [16]:
print(len(errors_list))
errors_list

0


[]

We don't get any errors, so that's good.

Now we can concatenate all the `DataFrame`s we scraped and create one large `DataFrame` containing all the passing data

In [17]:
type(passing_dfs_list)

list

In [18]:
passing_dfs_list[0:1]

[    Year                 0    1   2      3   4   5       6    7    8 ...  \
 0   1967  Sonny Jurgensen*  WAS  33     QB  14  14   5-6-3  288  508 ...   
 1   1967   Johnny Unitas*+  BAL  34     QB  14  14  11-1-2  255  436 ...   
 2   1967        Norm Snead  PHI  28     QB  14  14   6-7-1  240  434 ...   
 3   1967          Jim Hart  STL  23     QB  14  14   6-7-1  192  397 ...   
 4   1967   Fran Tarkenton*  NYG  27     QB  14  14   7-7-0  204  377 ...   
 5   1967    Roman Gabriel*  RAM  27     QB  14  14  11-1-2  196  371 ...   
 6   1967       John Brodie  SFO  32     QB  14  10   5-5-0  168  349 ...   
 7   1967     Randy Johnson  ATL  23     QB  14  12  1-10-1  142  288 ...   
 8   1967        Frank Ryan  CLE  31     QB  13  13   9-4-0  136  280 ...   
 9   1967          Kent Nix  PIT  23     QB  12   9   3-6-0  136  268 ...   
 10  1967       Gary Cuozzo  NOR  26     QB  13  10   3-7-0  134  260 ...   
 11  1967     Don Meredith*  DAL  29     QB  11  11   7-4-0  128  255 ...   

In [19]:
column_headers.insert(0, "Player_name")
column_headers.insert(0, "Year")
print(column_headers)
print(len(column_headers))

['Year', 'Player_name', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'NY/A', 'ANY/A', 'Sk%', '4QC', 'GWD']
30


In [20]:
# store all passing data in one DataFrame
passing_df = pd.concat(passing_dfs_list, axis=0)
#passing_df = passing_df.iloc[:,0:-1]
passing_df.columns = column_headers

In [21]:
# Take a look at the first few rows
passing_df.loc[0,:]

,Year,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,1967,Sonny Jurgensen*,WAS,33,QB,14,14,5-6-3,288,508,...,267.6,87.3,,,,,,4,2,NaN
0,1968,John Brodie,SFO,33,QB,14,14,7-6-1,234,404,...,215.7,78.0,,,,,,1,2,NaN
0,1969,Sonny Jurgensen*+,WAS,35,QB,14,14,7-5-2,274,442,...,221.6,85.4,40,322,5.77,5.28,8.3,2,0,NaN
0,1970,Roman Gabriel,RAM,30,QB,14,14,9-4-1,211,407,...,182.3,72.2,20,134,5.66,5.15,4.7,1,1,NaN
0,1971,John Hadl,SDG,31,QB,14,14,6-8-0,233,431,...,219.6,68.9,16,145,6.55,4.98,3.6,2,3,NaN
0,1972,Archie Manning,NOR,23,QB,14,14,2-11-1,230,448,...,198.6,64.6,43,347,4.96,3.77,8.8,1,1,NaN
0,1973,Roman Gabriel*,PHI,33,QB,14,14,5-8-1,270,460,...,229.9,86.0,31,219,6.11,5.95,6.3,3,2,NaN
0,1974,Jim Hart*,STL,30,QB,14,14,10-4-0,200,388,...,172.2,79.5,16,134,5.64,5.74,4.0,1,2,NaN
0,1975,Fran Tarkenton*+,MIN,35,QB,14,14,12-2-0,273,425,...,213.9,91.8,27,245,6.08,5.89,6.0,1,1,NaN
0,1976,Jim Zorn,SEA,23,QB,14,14,2-12-0,208,439,...,183.6,49.5,25,196,5.12,3.02,5.4,,,NaN


Now that we fixed up the necessary columns, let's write out the raw data to a CSV file.

In [22]:
if not os.path.exists('../data/raw_data'):
    os.makedirs('../data/raw_data')

if not os.path.exists('../data/clean_data'):
    os.makedirs('../data/clean_data')

In [23]:
passing_df.head()

,Year,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,1967,Sonny Jurgensen*,WAS,33,QB,14,14,5-6-3,288,508,...,267.6,87.3,,,,,,4,2,NaN
1,1967,Johnny Unitas*+,BAL,34,QB,14,14,11-1-2,255,436,...,244.9,83.6,,,,,,4,3,NaN
2,1967,Norm Snead,PHI,28,QB,14,14,6-7-1,240,434,...,242.8,80.0,,,,,,0,1,NaN
3,1967,Jim Hart,STL,23,QB,14,14,6-7-1,192,397,...,214.9,58.4,,,,,,2,2,NaN
4,1967,Fran Tarkenton*,NYG,27,QB,14,14,7-7-0,204,377,...,220.6,85.9,,,,,,2,2,NaN


In [24]:
# Write out the raw passing data to the raw_data fold in the data folder
passing_df.to_csv("../data/raw_data/pfr_nfl_passing_data_RAW.csv", index=False)

# Cleaning the Data

Now that we have the raw passing data, we need to clean it up a bit in order to do some of the data exploration we want.  

In [25]:
passing_df_clean = pd.read_csv("../data/raw_data/pfr_nfl_passing_data_RAW.csv")

* create a dictionary for renaming passing columns
* rename all columns with passing_columns dictionary

In [26]:
passing_columns = {
              'Year':'year', 'Player_name': 'name', 'Tm': 'team', 'Age': 'age',
              'Pos': 'position', 'G': 'games_played', 'GS':
              'games_started', 'QBrec': 'record', 'Cmp': 'completions',
              'Att': 'attempts', 'Cmp%': 'completionPct', 'Yds':
              'passing_yards', 'TD': 'passing_TD',  'TD%':
              'passing_TDPct', 'Int': 'passing_INT', 'Int%':
              'passing_INTPct', 'Lng': 'passing_long', 'Y/A':
              'passing_ydsAtt', 'AY/A': 'passing_airydsAtt', 'Y/C':
              'passing_ydsComp', 'Y/G': 'passing_ydsGame', 'Rate':
              'passing_rating', 'QBR': 'QBR','Sk': 'passing_sacks', 'Yds.1':
              'passing_sacksyds','NY/A': 'netydsAtt', 'ANY/A': 'passing_airnetydsAtt',
              'Sk%': 'passing_sackPct', '4QC': 'FourthQtrComebacks',
              'GWD': 'gamewinningdrives'
              }
ordered_columns = [
              'name', 'team',  'year', 'age', 'position', 'wins',
              'losses', 'games_played', 'games_started', 'completions',
              'attempts', 'completionPct', 'passing_yards',
              'passing_TD', 'passing_TDPct',  'passing_INT',
              'passing_INTPct', 'passing_long', 'passing_ydsAtt',
              'passing_airydsAtt', 'passing_ydsComp',
              'passing_ydsGame', 'passing_rating', 'passing_sacks',
              'passing_sacksyds', 'passing_airnetydsAtt',
              'passing_sackPct', 'FourthQtrComebacks',
              'gamewinningdrives'
    ]

In [27]:
passing_df_clean.rename(index=str,columns=passing_columns, inplace=True)
passing_df_clean.head()

,year,name,team,age,position,games_played,games_started,record,completions,attempts,...,passing_ydsGame,passing_rating,QBR,passing_sacks,passing_sacksyds,netydsAtt,passing_airnetydsAtt,passing_sackPct,FourthQtrComebacks,gamewinningdrives
0,1967,Sonny Jurgensen*,WAS,33,QB,14,14,5-6-3,288,508,...,267.6,87.3,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
1,1967,Johnny Unitas*+,BAL,34,QB,14,14,11-1-2,255,436,...,244.9,83.6,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN
2,1967,Norm Snead,PHI,28,QB,14,14,6-7-1,240,434,...,242.8,80.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN
3,1967,Jim Hart,STL,23,QB,14,14,6-7-1,192,397,...,214.9,58.4,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN
4,1967,Fran Tarkenton*,NYG,27,QB,14,14,7-7-0,204,377,...,220.6,85.9,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN


## Cleaning Up the Rest of the Passing Data

* process record columns into wins and losses

In [28]:
passing_df_clean['record'] = passing_df_clean['record'].astype('str')
passing_df_clean['position'] = passing_df_clean['position'].astype('str')
passing_df_clean['position'] = passing_df_clean['position'].str.upper()
passing_df_clean['record'].loc[passing_df_clean.record == 'QBrec'] = "0-0-0"
passing_df_clean['record'].loc[passing_df_clean.record == 'nan'] = "0-0-0"
unformatted_record = passing_df_clean['record'].str[:]
unformatted_record = unformatted_record.str.replace("-",  "/")
passing_df_clean['wins'] = unformatted_record.str.split("/").str[0]
passing_df_clean['losses'] = unformatted_record.str.split("/").str[1]
passing_df_clean['name'] = passing_df_clean['name'].str.replace('[+|*]', "")
passing_df_clean['wins'] = passing_df_clean['wins'].astype('float')
passing_df_clean = pd.DataFrame(data=passing_df_clean, columns=ordered_columns)

/Users/kivan/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
# convert the data to proper numeric types
passing_df_clean = passing_df_clean.apply(pd.to_numeric, errors="ignore")
passing_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4901 entries, 0 to 4900
Data columns (total 29 columns):
name                    4901 non-null object
team                    4901 non-null object
year                    4901 non-null int64
age                     4901 non-null int64
position                4901 non-null object
wins                    4901 non-null float64
losses                  4901 non-null int64
games_played            4901 non-null int64
games_started           4901 non-null int64
completions             4901 non-null int64
attempts                4901 non-null int64
completionPct           4901 non-null float64
passing_yards           4901 non-null int64
passing_TD              4901 non-null int64
passing_TDPct           4901 non-null float64
passing_INT             4901 non-null int64
passing_INTPct          4901 non-null float64
passing_long            4901 non-null int64
passing_ydsAtt          4901 non-null float64
passing_airydsAtt       4901 non-null float64
pas

We are not done yet. A lot of out numeric columns are missing data because players didn't accumulate any of those stats.  For example, some players didn't score a TD or even play a game.  Let's select the columns with numeric data and then replace the `NaN`s (the current value that represents the missing data) with 0s, as that is a more appropriate value. 

In [30]:
# Get the column names for the numeric columns
num_cols = passing_df_clean.columns[passing_df_clean.dtypes != object]

# Replace all NaNs with 0
passing_df_clean.loc[:, num_cols] = passing_df_clean.loc[:, num_cols].fillna(0)

In [31]:
# Everything is filled, except for Player_ID, which is fine for now
passing_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4901 entries, 0 to 4900
Data columns (total 29 columns):
name                    4901 non-null object
team                    4901 non-null object
year                    4901 non-null int64
age                     4901 non-null int64
position                4901 non-null object
wins                    4901 non-null float64
losses                  4901 non-null int64
games_played            4901 non-null int64
games_started           4901 non-null int64
completions             4901 non-null int64
attempts                4901 non-null int64
completionPct           4901 non-null float64
passing_yards           4901 non-null int64
passing_TD              4901 non-null int64
passing_TDPct           4901 non-null float64
passing_INT             4901 non-null int64
passing_INTPct          4901 non-null float64
passing_long            4901 non-null int64
passing_ydsAtt          4901 non-null float64
passing_airydsAtt       4901 non-null float64
pas

We are finally done cleaning the data and now we can save it to a CSV file.

In [32]:
passing_df_clean = passing_df_clean[pd.notnull(passing_df_clean['name'])]
passing_df_clean.sort_values('wins', ascending=False, axis=0, inplace=True)


In [34]:
passing_df_clean.to_csv("../data/clean_data/pfr_nfl_passing_data_CLEAN.csv", index=False)
passing_df_clean.head()

,name,team,year,age,position,wins,losses,games_played,games_started,completions,...,passing_airydsAtt,passing_ydsComp,passing_ydsGame,passing_rating,passing_sacks,passing_sacksyds,passing_airnetydsAtt,passing_sackPct,FourthQtrComebacks,gamewinningdrives
3985,Tom Brady,NWE,2007,30,QB,16.0,0,16,16,398,...,9.4,12.1,300.4,117.2,21.0,128.00,8.88,3.5,4.0,4.0
4779,Cam Newton,CAR,2015,26,QB,15.0,1,16,16,296,...,8.3,13.0,239.8,99.4,33.0,284.00,7.20,6.3,3.0,4.0
3556,Tom Brady,NWE,2003,26,QB,14.0,2,16,16,317,...,6.7,11.4,226.3,85.9,219.0,6.08,5.70,4.0,7.0,0.0
1592,Dan Marino,MIA,1984,23,QB,14.0,2,16,16,362,...,9.4,14.0,317.8,108.9,120.0,8.60,2.30,2.0,3.0,0.0
3898,Philip Rivers,SDG,2006,25,QB,14.0,2,16,16,284,...,7.4,11.9,211.8,92.0,27.0,144.00,6.73,5.5,4.0,4.0
